<a href="https://colab.research.google.com/github/YeongRoYun/BearTeam/blob/yyr-dev-data-eda/data/eda/EDA01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-HUB Dataset EDA
> ` https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=629`


In [3]:
import os
from pathlib import Path
basePath = Path(os.getcwd()).parent
dataPath = basePath / 'datasets'

assert os.path.exists(dataPath)

## 파일구조 확인하기
- Raw Image와 동일한 폴더 구조에 각 Image의 Annotation JSON File이 위치한다.
- File Structure

```
datasets
   |
   | --- Training/Validation
           |
           | ---- Annotations(라벨링데이터)/Raws(원천데이터)
                        |
                        | ---- BoundingBox(...2D BB)
                                  |
                                  | --- Day(주간)/Night(야간)
                                            |
                                            | ---- Clear(맑음)/Rain(비)/Snow(눈)
                                                            |
                                                            | ---- OldTown(구도심)
                                                            | ---- NewTown(신도심)
                                                            | ---- Alley(골목)
                                                            | ---- Country(시골길)
                                                            | ---- Mountain(산길)
                                                            | ---- Coastal(해안도로)
                                                            | ---- Highway(고속도로)
                                                            | ---- Motorway(자동차전용)
                                                                |
                                                                | dates(yyyymmdd_serial)
                                                                    |-- Camera
                                                                      |
                                                                      | -- front
                                                                      | -- left
                                                                      | -- rear
                                                                      | -- right
                                                                      | -- stereo_l
                                                                      | -- stereo_r
                                                                         |
                                                                         | -- id.json/id.jpg
```
- 데이터셋은 위의 파일 구조 형태를 공유한다.

## 파일구조 통일시키기
- 한글과 영어가 혼용된 파일명을 영어로 일관되게 통일시킨다.
- 예시: `주간 -> Day, 맑음 -> Sun`

> Annotation data로 Formating을 검증한다.

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import re

In [7]:
# Training / Validation relative Path
trainingPath = 'Training'
validationPath = 'Validation'

assert os.path.exists(dataPath / trainingPath)
assert os.path.exists(dataPath / validationPath)

# Annotations/Raws relative Path
rawPath = Path('Raws')
annotationPath = Path('Annotations')

assert os.path.exists(dataPath / trainingPath / annotationPath)
assert os.path.exists(dataPath / validationPath / annotationPath)

# BoundingBox relative Path
boundingBoxPath = Path('boundingBox')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath)
assert os.path.exists(dataPath / validationPath / annotationPath / boundingBoxPath)


# Day/ Night relative path
dayPath = Path('Day')
nightPath = Path('Night')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath)
assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / nightPath)

# Sun/Rain/Snow relative Path(없을 수도 있다.)
clearPath = Path('Clear')
rainPath = Path('Rain')
snowPath = Path('Snow')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath)
assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / rainPath)

# Road relative Path(없을 수도 있다.)
oldTownPath = Path('OldTown')
newTownPath = Path('NewTown')
alleyPath = Path('Alley')
countryPath = Path('Country')
mountainPath = Path('Mountain')
coastalPath = Path('Coastal')
highwayPath = Path('Highway')
motorwayPath = Path('Motorway')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath)
assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / newTownPath)
assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / motorwayPath)

# Date relative Path(20220101_001)
datePathRegex = r'(\d{8}_\d{3})'

testPath = dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath

datePaths = [d.name for d in testPath.iterdir()]
print(f"date: {datePaths[0]}")

# Camera relative Path
cameraPath = Path('Camera')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath / datePaths[0] / cameraPath)

# Camera Position relative Path(없을 수 있다!)
frontPath = Path('front')
rearPath = Path('rear')
leftPath = Path('left')
rightPath = Path('right')
stereo_lPath = Path('stereo_l')
stereo_rPath = Path('stereo_r')

assert os.path.exists(dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath / '20210906_018' / cameraPath / frontPath)

# Id relative path
idPathRegex = r'(.+).json'

testPath = dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath / '20210906_018' / cameraPath / frontPath
idPaths = [i.name for i in testPath.iterdir()]
print(f"id: {idPaths[0]}")
print(f"extract id: {re.search(idPathRegex, '1_1_1_20210906_018_00000251.json')[1]}")


date: 20211130_038
id: 1_1_1_20210906_018_00000081.json
extract id: 1_1_1_20210906_018_00000251


## JSON 형식 확인
```
{
    "Images.identifier"    : Raws 이미지와 Annotations Json을 연결하는 id
    "Images.type"          : Image의 확장자,
    "Images.width"         : Image의 가로 길이
    "Images.height"        : Image의 세로 길이
    "Images.data_captured" : 이미지가 촬영된 시간(yyyy-mm-dd hh:mm:ss)
    "Images.weather"       : 이미지가 촬영된 날씨(clear/rain/snow)
    "Images.road_type"     : 이미지가 촬영된 도로
                           : urban_old(구도심), urban_new(신도심), residential(주거지역), rural(시골길)
                           : mountainous(산길), coastal(해안도로), highway(고속도로), car_only(자동차 전용도로)
    "Images.frame_num"     : 영상 내 프레임 위치
    "Images.num_object"    : 이미지 내 객체 수
    "Images.num_trackid"   : Sequence에서 추적하고 있는 대상의 수
    "annotations"          : 라벨링 정보
          {
               "bbox.id"        : 바운딩박스 식별자
               "bbox.category"  : 바운딩박스 물체 분류
                                : "car", "truck", "bus", "other vehicles", "pedestrian"(보행자)
                                : "motorcycle", "bicycle", "dontcare"
                                
               "bbox.x"         : 바운딩박스 시작점(min_x)
               "bbox.y"         : 바운딩박스 시작점(min_y)
               "bbox.width"     : 바운딩박스 가로 길이
               "bbox.height"    : 바운딩박스 세로 길이
               "bbox.size"      : 바운딩 박스 크기 정도(small/medium/large)
               "bbox.occluded"  : 객체가 가려졌는지(0: 온전함, 1: 가려짐(~50%), 2: 가려짐(~70%))
               "bbox.truncated" : 객체가 잘렸는지(0: 온전함, 1: 잘림)
          }
}
```

### 신뢰할 수 없는 정보
- `bbox.size`
  - 비슷한 비율의 박스가 Small/Medium으로 혼동되어 분류된다.
  - 일정한 비율을 정하여 새롭게 라벨링 해야만 한다.
  - 기준(area_ratio)
    - `small   : 0.0 ~ 0.01`
    - `medium  : 0.01 ~ 0.04`
    - `large   : 0.04 ~ 1.0`

In [9]:
testPath = testPath = dataPath / trainingPath / annotationPath / boundingBoxPath / dayPath / clearPath / oldTownPath / '20210906_018' / cameraPath / frontPath
testJson = [i.name for i in testPath.iterdir()][0]

testJsonPath = testPath / testJson

with open(testJsonPath, 'r') as fd:
    test = json.load(fd)
    print(json.dumps(test, indent=4))
    
    # 파일의 id = Images.identifier = raws iamage의 name이다.
    assert test['Images.identifier'] == re.search(idPathRegex, testJson)[1]
    
    annotations = test['annotations']
    
    # Bounding Box는 num_object만큼 있다.
    assert len(annotations) == test['Images.num_object']
    
    print("\n\n======================\n")
    
    # bbox size 확인
    # None 값이 포함된다.
    # 신뢰할 수 없는 분류를 가진다.
    for annotation in annotations:
        width_ratio = annotation['bbox.width'] / test['Images.width']
        height_ratio = annotation['bbox.height'] / test['Images.height']
        print(f"size: {annotation['bbox.size']}, area_ratio: {width_ratio * height_ratio:.2f}")
        
    
    print("\n\n======================\n")
    # Re-Labeling
    bboxSizeClassifier = {'small': 0.01, 'medium': 0.04, 'large': 1.0}
    for annotation in annotations:
        width_ratio = annotation['bbox.width'] / test['Images.width']
        height_ratio = annotation['bbox.height'] / test['Images.height']
        area_ratio = width_ratio * height_ratio
        
        if area_ratio <= bboxSizeClassifier['small']:
            annotation['bbox.size'] = 'small'
        elif area_ratio < bboxSizeClassifier['medium']:
            annotation['bbox.size'] = 'medium'
        else:
            annotation['bbox.size'] = 'large'
        
        print(f"size: {annotation['bbox.size']}, area_ratio: {width_ratio * height_ratio:.2f}")   
    

{
    "Images.identifier": "1_1_1_20210906_018_00000081",
    "Images.type": "jpg",
    "Images.width": 1920,
    "Images.height": 1200,
    "Images.data_captured": "2021-09-06 11:45:32",
    "Images.weather": "clear",
    "Images.road_type": "urban_old",
    "Images.frame_num": 81,
    "Images.num_object": 5,
    "Images.num_trackid": 430,
    "annotations": [
        {
            "bbox.id": 1,
            "bbox.category": "car",
            "bbox.x": 901.1708984375,
            "bbox.y": 636.5517578125,
            "bbox.width": 138.02910156250073,
            "bbox.height": 111.14824218750073,
            "bbox.size": "medium",
            "bbox.truncated": 0,
            "bbox.occluded": 0
        },
        {
            "bbox.id": 13,
            "bbox.category": "truck",
            "bbox.x": 1053.041015625,
            "bbox.y": 574.69921875,
            "bbox.width": 54.069726562500364,
            "bbox.height": 84.40078125000036,
            "bbox.size": "medium",
         

## Annotation 이용 방법

### 보다 정밀한 Loss 계산
- `truncated`와 `occluded` 정보를 활용해 가중치를 줌으로써 더 정확하게 계산할 수 있도록 만든다.
- `wheather` 정보를 활용해 가중치를 줌으로써 더 정확하게 계산할 수 있도록 만든다.

### 사용할 Object Class
- 자전거 전용 도로에서 자전거/오토바이/사람의 후방에서의 추월을 탐지하는 것이 본 모델의 목적이다.
- 따라서 "motorcycle", "bicycle", "pedestrian" Class를 제외한 나머지는 "dontcare"로 설정하여 하나의 Class로 합친다.

### Data Inbalance
- 위에서 Class를 합치는 경우 데이터 불균형 문제가 커질 것이다.
- 프로젝트 마감이 얼마 남지 않은 관계로 우리는 Downsampling을 사용해 보다 적은 Class에 맞춰 데이터의 비중을 조정할 것이다.

# Dataset 구축하기

In [82]:
# Dataset 만들기
from torch.utils.data import Dataset
from pathlib import Path
import os
from functools import reduce

class CustomYoloDataset(Dataset):
    """
    Raw Image와 Annotations를 정제한다.
    """
    def __init__(self, root="datasets/", train=True):
        """
        path : 데이터셋이 위치한 경로
        train : train/validation Dataset 만들기!
        """
        super().__init__()
        self._setPaths(root)
        self._train = train
        self._len = self._get_len()
        
    def _setPaths(self, root):
        """
        Path object로 path를 지정한다.
        """
        self._rootPath = Path(root)
        self._trainingPath = 'Training'
        self._validationPath = 'Validation'
        
        # Raw / Annotations
        self._rawPath = Path('Raws')
        self._annotationPath = Path('Annotations')
        
        # BBOX
        self._boundingBoxPath = Path('boundingBox')
        
        # 아침/밤
        self._dayPath = Path('Day')
        self._nightPath = Path('Night')
        
        # 날씨
        self._clearPath = Path('Clear')
        self._rainPath = Path('Rain')
        self._snowPath = Path('Snow')
        
        # 도로
        self._oldTownPath = Path('OldTown')
        self._newTownPath = Path('NewTown')
        self._alleyPath = Path('Alley')
        self._countryPath = Path('Country')
        self._mountainPath = Path('Mountain')
        self._coastalPath = Path('Coastal')
        self._highwayPath = Path('Highway')
        self._motorwayPath = Path('Motorway')
        
        # 날짜는 정규식으로 대체한다.
        
        # 카메라
        self._cameraPath = Path('Camera')
        self._frontPath = Path('front')
        self._rearPath = Path('rear')
        self._leftPath = Path('left')
        self._rightPath = Path('right')
        self._stereo_lPath = Path('stereo_l')
        self._stereo_rPath = Path('stereo_r')
        
        
    def __getitem__(self, idx):
        """
        Return : Raw Image, Annotation
        순서는 위의 Path가 정해진 순서로 반환한다.
        """
        # 일단 Annotation만 반환!
        raws, annotations = self._get_files()
        
        # 각 folder의 길이를 구한다.
        counts = list(map(lambda path: len(os.listdir(path)), annotations))
        
        # index가 위치할 폴더를 찾는다.
        folderIdx = 0
        for idx, count in enumerate(counts):
            if idx - count < 0:
                folderIdx = idx
                break
            else:
                idx -= count
        
        try:
            annotationPath = annotations[folderIdx]
            annotationPath = annotationPath / os.listdir(annotationPath)[idx]
            
            rawFd = None
            annotationFd = open(annotationPath)
            
        except FileNotFoundError as e:
            raise e
        except StopIteration as e:
            pass
        else:
            return rawFd, json.load(annotationFd)
        
    
    
    def __len__(self):
        return self._len
    
    
    def _get_len(self):
        """
        카메라 폴더(front/rear...)에 있는 annotation file의 갯수를 반환한다.
        """
        _, annotations = self._get_files()
        # annotation file 갯수 더하기!
        count = reduce(lambda x, path: x + len(os.listdir(path)) if path.exists() else x, annotations, 0)
        return count
        
    
    def _concat_pathes(self, bases, relatives):
        """
        bases과 relatives의 모든 조합을 반환한다.
        """
        tmp = []
        for base in bases:
            tmp.extend([base / rel for rel in relatives])
        return tmp
    
    def _get_files(self):
        """
        실제 사용할 File들의 Path를 반환한다.
        실제 존재하는 파일인지도 여기에서 1차적으로 확인!!
        """
        # 일단 Annotation부터!
        
        if self._train:
            path = self.trainingPath
        else:
            path = self.validationPath
        
        path = path / self._annotationPath / self._boundingBoxPath
        
        #낮/밤
        pathes = [path / self._dayPath, path / self._nightPath]
        
        
        #날씨
        pathes = self._concat_pathes(pathes, [self._clearPath, self._rainPath, self._snowPath])
        
        # 도로
        pathes = self._concat_pathes(pathes, [self._oldTownPath, self._newTownPath, self._alleyPath, \
                                              self._countryPath, self._mountainPath, self._coastalPath,\
                                              self._highwayPath, self._motorwayPath])
        # 날짜
        tmp = []
        for path in pathes:
            if path.exists():
                dates = [date for date in path.iterdir()]
                tmp.extend(dates)
        pathes = tmp

        # 카메라
        pathes = list(map(lambda path: path / self._cameraPath, pathes))
        pathes = self._concat_pathes(pathes, [self._frontPath, self._rearPath, self._leftPath, \
                                              self._rightPath, self._stereo_lPath, self._stereo_rPath])
        
        # 진짜 있는지 필터링
        pathes = list(filter(lambda path: path.exists(), pathes))
        return None, pathes
    
    @property
    def rootPath(self):
        return self._rootPath
    @property
    def trainingPath(self):
        return self._rootPath / self._trainingPath
    @property
    def validationPath(self):
        return self._rootPath / self._validationPath

    def is_train(self):
        """
        Dataset이 Train/Valid인지 확인하기
        """
        return self._train